<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment3/blob/master/src/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

--2021-04-26 11:43:32--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 142.251.5.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  33.0MB/s    in 17s     

2021-04-26 11:43:49 (111 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [2]:
!tar xopf dakshina_dataset_v1.0.tar

In [3]:
!ls dakshina_dataset_v1.0/hi/lexicons

hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [4]:
import numpy as np

def read_data(data_path, characters = False):
    # Returns the (input, output) pair from the dataset
    # If characters == True, the input/output would be in the form list of characters, else as string

    with open(data_path, "r", encoding="utf-8") as f:
        lines = [line.split("\t") for line in f.read().split("\n") if line != '']
    
    input, target = [val[1] for val in lines], [val[0] for val in lines]
    if characters:
        input, target = [list(inp_str) for inp_str in input], [list(tar_str) for tar_str in target]
    return input, target


def process_data(input, output, enc_timesteps, dec_timesteps, input_char_enc, target_char_enc):
    # Returns the input and output data in a form needed by the Keras embedding layer, where each character is encoded by an integer

    # ' ' -- space (equivalent to no meaningful input)
    encoder_input = np.array([[input_char_enc[ch] for ch in string] + [input_char_enc[' ']] * (enc_timesteps - len(string)) for string in input])
    # '\t' -- start of sequence, '\n' -- end of sequence
    decoder_input = np.array([[target_char_enc['\t']] + [target_char_enc[ch] for ch in string] + [target_char_enc['\n']] 
                                 + [target_char_enc[' ']] * (dec_timesteps - len(string) - 2) for string in output])
    decoder_target = np.zeros((decoder_input.shape[0], dec_timesteps, len(target_char_enc)), dtype='float32')

    for i in range(decoder_input.shape[0]):
        for t, char_ind in enumerate(decoder_input[i]):
            if t > 0:
                decoder_target[i,t-1,char_ind] = 1.0
        decoder_target[i,t:,target_char_enc[' ']] = 1.0

    return encoder_input, decoder_input, decoder_target


def encode_decode_characters(train_input, train_target, val_input, val_target):
    # Returns the encoder for characters to integer (as a dictionary) and decoder for integers to characters (as a list) for input and target data

    input_char_enc = {}
    input_char_dec = []
    max_encoder_seq_length = 1
    for string in train_input + val_input:
        max_encoder_seq_length = max(max_encoder_seq_length, len(string))
        for char in string:
            if char not in input_char_enc:
                input_char_enc[char] = len(input_char_dec)
                input_char_dec.append(char)
    if ' ' not in input_char_enc:
        input_char_enc[' '] = len(input_char_dec)
        input_char_dec.append(' ')

    target_char_enc = {}
    target_char_dec = []
    target_char_enc['\t'] = len(target_char_dec)
    target_char_dec.append('\t')
    max_decoder_seq_length = 1
    for string in train_target + val_target:
        max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
        for char in string:
            if char not in target_char_enc:
                target_char_enc[char] = len(target_char_dec)
                target_char_dec.append(char)
    target_char_enc['\n'] = len(target_char_dec)
    target_char_dec.append('\n')
    if ' ' not in target_char_enc:
        target_char_enc[' '] = len(target_char_dec)
        target_char_dec.append(' ')

    print("Number of training samples:", len(train_input))
    print("Number of validation samples:", len(val_input))
    print("Number of unique input tokens:", len(input_char_dec))
    print("Number of unique output tokens:", len(target_char_dec))
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

    return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length



In [7]:
input_char_enc = {}
input_char_dec = []
target_char_enc = {}
target_char_dec = []
max_encoder_seq_length = 0
max_decoder_seq_length = 0

# Reading training and validation data
train_inp, train_out = read_data('./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv', True)
val_inp, val_out = read_data('./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv', True)
# Assigning encoder and decoder for input and target characters
input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length = encode_decode_characters(
    train_inp, train_out, val_inp, val_out)

# Assigning training and validation encoder input, decoder input, decoder output
train_enc_input, train_dec_input, train_dec_target = process_data(train_inp, train_out, max_encoder_seq_length, max_decoder_seq_length, 
                                                                  input_char_enc, target_char_enc)
val_enc_input, val_dec_input, val_dec_target = process_data(val_inp, val_out, max_encoder_seq_length, max_decoder_seq_length, 
                                                            input_char_enc, target_char_enc)


Number of training samples: 44204
Number of validation samples: 4358
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [8]:
import tensorflow as tf
import tensorflow.keras as keras

def create_model(encoder_vocab_size, decoder_vocab_size, encoder_timesteps, decoder_timesteps,
                 inp_emb_size=16, no_enc_layers=1, no_dec_layers=1, hid_layer_size=32, cell_type='LSTM', dropout=0, cell_activation='tanh'):
    
    get_cell = {
        'RNN': keras.layers.SimpleRNN,
        'GRU': keras.layers.GRU,
        'LSTM': keras.layers.LSTM
    }
    # Encoder input and embedding
    encoder_input = keras.layers.Input(shape=(encoder_timesteps,))
    encoder_inp_emb = keras.layers.Embedding(encoder_vocab_size, inp_emb_size, input_length=encoder_timesteps)(encoder_input)

    # Encoder cell layers
    encoder_seq, *encoder_state = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                      recurrent_dropout=dropout, name="encoder_1")(
                                                            encoder_inp_emb
                                                     )
    for i in range(1, no_enc_layers):
        encoder_seq, *encoder_state = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                          recurrent_dropout=dropout, name="encoder_"+str(i+1))(
                                                                encoder_seq
                                                         )
    
    # Decoder input and embedding
    decoder_input = keras.layers.Input(shape=(decoder_timesteps,))
    decoder_inp_emb = keras.layers.Embedding(decoder_vocab_size, inp_emb_size, input_length=decoder_timesteps)(decoder_input)

    # Decoder cell layers
    decoder_seq, *_ = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                          recurrent_dropout=dropout, name="decoder_1")(
                                                decoder_inp_emb, initial_state=encoder_state
                                         )
    for i in range(1, no_dec_layers):
        decoder_seq, *_ = get_cell[cell_type](hid_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                              recurrent_dropout=dropout, name="decoder_"+str(i+1))(
                                                    decoder_seq, initial_state=encoder_state
                                             )
    
    # Softmax FC layer
    decoder_output = keras.layers.Dense(decoder_vocab_size, activation="softmax")(
        decoder_seq
    )

    # Define the model that will turn encoder_input_data and decoder_input_data into decoder_target_data
    model = keras.Model([encoder_input, decoder_input], decoder_output)

    model.summary()
    return model

In [9]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 11.0MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 133kB 37.6MB/s 
     |████████████████████████████████| 163kB 35.9MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7e91965770ee287b0fa1c4152c59b935ff8f1c8da2335f44cf573210e6e7d641
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=3016c688fab3d7f334da44c5d863f730e817b3d80c64f953dd564cc118cab512
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

def train_model(model, train_input_data, train_target_data, val_input_data, val_target_data, 
                batch_size=64, optimizer = 'adam', learning_rate = 1e-3, epochs = 10):
    # Function to train the model using the mentioned optimizer, learning rate and epochs using given training and validation data

    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=learning_rate), loss = "categorical_crossentropy", metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9), loss = "categorical_crossentropy", metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=learning_rate), loss = "categorical_crossentropy", metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9, nesterov = True), loss = "categorical_crossentropy", metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=learning_rate), loss = "categorical_crossentropy", metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=learning_rate), loss = "categorical_crossentropy", metrics = ['accuracy'])

    # Using validation accuracy as the metric to monitor as that is what is intended to be maximized
    model.fit(train_input_data,
              train_target_data,
              batch_size = batch_size,
              epochs = epochs, 
              validation_data = (val_input_data, val_target_data),
              verbose = 2,
              callbacks = [WandbCallback(monitor='val_accuracy'), EarlyStopping(monitor='val_accuracy', patience=5)])

    return model

In [19]:
###### TO BE DONE : config dictionary, Seq2Seq Function to call the create_model and train_model
config_1 = {
    "learning_rate": 1e-3,                                      # Hyperparameter for updating the parameters in gradient descent
    "epochs": 10,                                               # Number of epochs to train the model   
    "optimizer": 'adam',                                    # Gradient descent algorithm used for the parameter updation
    "batch_size": 64,                                           # Batch size used for the optimizer
    "loss_function": 'categorical_crossentropy',                # Loss function used in the optimizer
    "architecture": 'RNN',                                      # Type of neural network used
    "dataset": "Dakshina",                                      # Name of dataset
    "inp_emb_size": 16,                                         # Number of filters for the first convolution layer
    "no_enc_layers": 1,                                         # The factor by which the number of filters change in the subseqeuent convolution layers
    "no_dec_layers": 1,                                         # Number of neurons in the dense FC layer
    "hid_layer_size": 16,                                       # True : Data augmentation is done during training, False : No data augmentation done
    "dropout" : 0.25,                                           # Probability of dropping out a neuron in dropout technique
    "cell_type": 'RNN',
    "beam": 1,
}


# run = wandb.init(project="assignment3", entity="abisheks", reinit=True)
# tf.keras.backend.clear_session()
# model = create_model(len(input_char_dec), len(target_char_dec), max_encoder_seq_length, max_decoder_seq_length)
# model = train_model(model, [train_enc_input,train_dec_input], train_dec_target, [val_enc_input,val_dec_input], val_dec_target)
# run.finish()

def RNN_train(config):
  model = create_model(len(input_char_dec), len(target_char_dec), max_encoder_seq_length, max_decoder_seq_length, config['inp_emb_size'],
                       config['no_enc_layers'], config['no_dec_layers'], config['hid_layer_size'], config['cell_type'], config['dropout']
                       )
  model = train_model(model, [train_enc_input,train_dec_input], train_dec_target, [val_enc_input,val_dec_input], val_dec_target, 
                      config_1['batch_size'], config['optimizer'])
  return model

In [20]:
#print([layer.name for layer in model.layers])

In [21]:
def inference_model(model):
    encoder_input = model.input[0]  # input_1
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_input = model.input[1]  # input_2
    decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
    decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_lstm = model.layers[3]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[4]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )

In [22]:
# Hyperparameter choices to sweep 
sweep_config = {
    'name': 'RNN',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'inp_emb_size': {
            'values': [16, 32, 64, 256]
        },
        'no_enc_layers': {
            'values': [1, 2, 3]
        },
        'no_dec_layers': {
            'values': [1, 2, 3]
        },
        'hid_layer_size': {
            'values': [16, 32, 64, 256]
        },
        'cell_type': {
            'values': ['RNN', 'LSTM', 'GRU']
        },
        'dropout' :{
            'values': [0, 0.25, 0.4]
        },
        'optimizer': {
            'values': ['adam', 'nesterov']
        },
        'beam': {
            'values': [1, 5]
        }
    }
}

In [25]:
def sweep_wrapper():
    # Wrapper function to call the CNN function for sweeping with different hyperparameters

    # Initialize a new wandb run
    run = wandb.init(config=config_1, reinit=True)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    wandb.run.name = f'ie_{config.inp_emb_size}_ne_{config.no_enc_layers}_de_{config.no_dec_layers}_ct_{config.cell_type}_dr_{config.dropout}'
    wandb.run.name += f'_da_{config.hid_layer_size}'
    wandb.run.save()
    print(wandb.run.name)

    modelA = RNN_train(config)
    run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="abisheks", project="assignment3")
wandb.agent(sweep_id, lambda : sweep_wrapper())

Create sweep with ID: vn44dqyq
Sweep URL: https://wandb.ai/abisheks/assignment3/sweeps/vn44dqyq


wandb: Agent Starting Run: shz1s4ft with config:
wandb: 	beam: 1
wandb: 	cell_type: GRU
wandb: 	dropout: 0
wandb: 	hid_layer_size: 256
wandb: 	inp_emb_size: 256
wandb: 	no_dec_layers: 1
wandb: 	no_enc_layers: 3
wandb: 	optimizer: nesterov


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


ie_256_ne_3_de_1_ct_GRU_dr_0_da_256
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 256)      6912        input_1[0][0]                    
__________________________________________________________________________________________________
encoder_1 (GRU)                 [(None, 20, 256), (N 394752      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________

epoch,9
loss,1.132
accuracy,0.71888
val_loss,1.08173
val_accuracy,0.73282
_runtime,271
_timestamp,1619438777
_step,9
best_val_accuracy,0.73282
best_epoch,8


...

epoch,9
loss,1.13964
accuracy,0.71721
val_loss,1.08528
val_accuracy,0.7327
_runtime,3074
_timestamp,1619450397
_step,9
best_val_accuracy,0.7327
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▂▁▁▁
accuracy,▁▂▃▄▅▆▇███
val_loss,█▆▅▄▃▂▂▁▁▁
val_accuracy,▁▃▃▄▅▆▇███
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 96ctfpsj with config:
wandb: 	beam: 1
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.4
wandb: 	hid_layer_size: 256
wandb: 	inp_emb_size: 256
wandb: 	no_dec_layers: 2
wandb: 	no_enc_layers: 2
wandb: 	optimizer: adam


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


ie_256_ne_2_de_2_ct_LSTM_dr_0.4_da_256
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 256)      6912        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
encoder_1 (LSTM)                [(None, 20, 256), (N 525312      embedding[0][0]                  
_______________________________________________________